In [1]:
import train as t
import importlib
importlib.reload(t)

import tensorflow as tf

import importlib

import numpy as np
from enum import Enum

from functools import reduce

In [2]:
class DS(Enum):
    ENCODER_INPUTS = 0
    TARGET_TEXTS = 1
    LENGTHS = 2
    DECODER_INPUTS = 3
    DECODER_TARGETS = 4
    

In [3]:
# Create the vocabulary
token_vocabulary = ["**end**", "**start**", "**unknown**"]

token_vocabulary.extend(t.get_vocabulary("train"))

target_tokens = token_vocabulary  # TODO: Refactor this. Currently duplicate naming

token_vocab_size = len(target_tokens)
# todo: document what was lifted from tutorials and what we wrote ourselves
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_target_token_index = dict(
    (i, char) for char, i in target_token_index.items())



print("\n ======================= Loading Data =======================")
#new cell

train_dataset = t.get_data_somehow('train(40, 160, 1)', False, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_train_num_samples'] , target_token_index)
val_dataset = t.get_data_somehow('val(40, 160, 1)', False, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_val_num_samples'], target_token_index)

train_encoder_input_data_batches = train_dataset[0]
train_target_texts_batches = train_dataset[1]
train_sequence_lengths_batches = train_dataset[2]
train_decoder_input_data_batches = train_dataset[3]
train_decoder_target_data_batches = train_dataset[4]

val_encoder_input_data_batches = val_dataset[0]
val_target_texts_batches = val_dataset[1]
val_sequence_lengths_batches = val_dataset[2]
val_decoder_input_data_batches = val_dataset[3]
val_decoder_target_data_batches = val_dataset[4]
print("\n ======================= Data Loaded =======================")


 ======================= Loading Data =======================
yesterday's data half price
loading:  train(40, 160, 1)
yesterday's data half price
loading:  val(40, 160, 1)

 ======================= Data Loaded =======================


In [4]:
tf.reset_default_graph()



g = t.create_graph(token_vocab_size, t.hparams['num_units'], t.hparams['use_attention'], t.hparams['use_encoding_average_as_initial_state'], False)
embedding_decoder = g['embedding_decoder']
decoder_cell = g['decoder_cell']
decoder_initial_state = g['decoder_initial_state']
projection_layer = g['projection_layer']
img = g['img']
decoder_lengths = g['decoder_lengths']
decoder_inputs = g['decoder_inputs']
decoder_outputs = g['decoder_outputs']
train_loss = g['train_loss']


INFO:tensorflow:Summary name conv2d/kernel:0/gradient is illegal; using conv2d/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d/bias:0/gradient is illegal; using conv2d/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_1/kernel:0/gradient is illegal; using conv2d_1/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d_1/bias:0/gradient is illegal; using conv2d_1/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0/gradient is illegal; using conv2d_2/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d_2/bias:0/gradient is illegal; using conv2d_2/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_3/kernel:0/gradient is illegal; using conv2d_3/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d_3/bias:0/gradient is illegal; using conv2d_3/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_4/kernel:0/gradient is illegal; using conv2d_4/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2

In [5]:
len(train_encoder_input_data_batches)

577

In [6]:
sess = tf.Session()
t.initialize_variables(sess, restore=True, path="/Users/adamjensen/project-environments/handwriting-to-latex-env/output/checkpoints/model_47.ckpt")



restoring
INFO:tensorflow:Restoring parameters from /Users/adamjensen/project-environments/handwriting-to-latex-env/output/checkpoints/model_47.ckpt


In [7]:
t_loss = t.get_loss(img,
                  train_encoder_input_data_batches,
                  decoder_lengths,
                  train_sequence_lengths_batches,
                  decoder_inputs,
                  train_decoder_input_data_batches,
                  decoder_outputs,
                  train_decoder_target_data_batches,
                  train_loss,
                  sess)

t_loss


17.205822449622989

In [8]:
v_loss = t.get_loss(img,
                  val_encoder_input_data_batches,
                  decoder_lengths,
                  val_sequence_lengths_batches,
                  decoder_inputs,
                  val_decoder_input_data_batches,
                  decoder_outputs,
                  val_decoder_target_data_batches,
                  train_loss,
                  sess)

v_loss


47.728467825687289

In [9]:
# val predictions
batch = val_encoder_input_data_batches[0]

little_variable, logits = t.predict_batch(sess,
                  batch,
                  target_token_index,
                  embedding_decoder,
                  decoder_cell,
                  decoder_initial_state,
                  projection_layer,
                  img)

logits.shape

(16, 70, 400)

In [ ]:
# train predictions
batch = train_encoder_input_data_batches[0]

little_variable, logits = t.predict_batch(sess,
                  batch,
                  target_token_index,
                  embedding_decoder,
                  decoder_cell,
                  decoder_initial_state,
                  projection_layer,
                  img)

logits.shape

In [10]:
def get_token_seq(int_sequence):
    
    output_string = ""
    for value in int_sequence:
        output_string += reverse_target_token_index[value] + " "
        if reverse_target_token_index[value] == "**end**":
            break
    return output_string

for idx, seq in enumerate(train_target_texts_batches[0]):
    
    print("Output: ")
    print(get_token_seq(little_variable[idx]))
    print("")
    print("Ground truth: ")
    print(train_target_texts_batches[0][idx])
    print("")

Output: 
E ( v ) = { \frac { d } { d \beta } } \, { \cal V } ( k ) **end** 

Ground truth: 
**start** \, ^ { * } d \, ^ { * } H = \kappa \, ^ { * } d \phi = J _ { B } .

Output: 
( \partial _ { i } , \partial _ { j } ) _ { ( n ) } = \sqrt { \chi } e ^ { 2 U } **end** 

Ground truth: 
**start** \partial _ { \mu } ( F ^ { \mu \nu } - e j ^ { \mu } x ^ { \nu } ) = 0 .

Output: 
{ \cal S } ^ { \mu \nu } \equiv \epsilon ^ { \mu \nu } \equiv \epsilon ^ { \mu \nu \nu } F _ { \mu } { } _ { \nu } = 0 \ \ , **end** 

Ground truth: 
**start** L _ { 0 } = \Phi ( w ) = \bigtriangleup \Phi ( w ) ,

Output: 
M _ { E } ^ { 2 } \int d c x ^ { - 2 \sigma ( x ) } **end** 

Ground truth: 
**start** \left( D ^ { * } D ^ { * } + m ^ { 2 } \right) { \cal H } = 0

Output: 
t _ { i } = { \frac { E } { 2 \pi } } , , \hspace { s c n } _ { 2 } = - \frac { 2 } { 2 \pi } , **end** 

Ground truth: 
**start** \langle T _ { z z } \rangle = - 3 \times \frac { \pi ^ { 2 } } { 1 4 4 0 a ^ { 4 } } .

Output: 
\chi _ { 2 }